In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd 
import random as rnd
from tqdm import tqdm_notebook #gives a progress bar

# visualizatoin
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
plt.style.use('ggplot')

# Models
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBRegressor

# Datasets
traintest = pd.read_csv('../input/predict-future-sales-feature-engineering/Traintest.csv')

train = pd.read_csv('../input/predict-future-sales-feature-engineering/Train_set.csv')
validation = pd.read_csv('../input/predict-future-sales-feature-engineering/Val_set.csv')
test = pd.read_csv('../input/predict-future-sales-feature-engineering/Test_set.csv')

## Overview

Check Datasets

In [2]:
print(train.shape)
train.head()

(6536234, 24)


,Month,date_block_num,item_category_id,item_cnt_month,item_id,item_price,m_num,shop_id,Revenue,year,...,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
0,Jan,0,2.0,9.0,5572,10730.0,1.0,2,96570.0,2013,...,0.0,9.0,9.0,0.11552,1.136364,1.060606,0.154522,0.208116,1590.135895,1505.450000
1,Jan,0,2.0,1.0,5643,2390.0,1.0,2,2390.0,2013,...,0.0,1.0,1.0,0.11552,1.785714,1.424242,0.154522,0.208116,4042.562819,3299.081818
2,Jan,0,5.0,2.0,5583,1188.3,1.0,2,2376.6,2013,...,0.0,2.0,2.0,0.11552,0.547619,0.242424,0.154522,0.208116,364.164069,163.100000
3,Jan,0,6.0,3.0,7893,5970.0,1.0,2,17910.0,2013,...,0.0,3.0,3.0,0.11552,2.975469,2.909091,0.154522,0.208116,5166.950289,5220.363636
4,Jan,0,6.0,1.0,7894,1490.0,1.0,2,1490.0,2013,...,0.0,1.0,1.0,0.11552,3.562771,1.484848,0.154522,0.208116,4874.250863,2264.818182


In [3]:
train.tail()

,Month,date_block_num,item_category_id,item_cnt_month,item_id,item_price,m_num,shop_id,Revenue,year,...,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
6536229,Sept,32,0.0,0.0,9103,0.0,9.0,36,0.0,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.272309,0.227376,0.0,0.0
6536230,Sept,32,0.0,0.0,9107,0.0,9.0,36,0.0,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.272309,0.227376,0.0,0.0
6536231,Sept,32,0.0,0.0,5704,0.0,9.0,36,0.0,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.272309,0.227376,0.0,0.0
6536232,Sept,32,0.0,0.0,12733,0.0,9.0,36,0.0,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.272309,0.227376,0.0,0.0
6536233,Sept,32,0.0,0.0,15925,0.0,9.0,36,0.0,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.272309,0.227376,0.0,0.0


Figured it out. Shop_id 36 is only in the validation set. So we created all of the previous items and shops for that data in the train set and made them equal zero. Therefore, all of the means for items and shops will be zero. The year and month will be the only ones with value. 

In [4]:
print(test.shape)
test.head()

(214200, 25)


,Unnamed: 0,Month,date_block_num,item_category_id,item_cnt_month,item_id,item_price,m_num,shop_id,Revenue,...,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
0,0,Nov,34,NaN,NaN,5037,NaN,11.0,5,NaN,...,0.0,1.2,1.000000,0.149369,0.807359,0.393939,0.272309,0.255791,1287.410133,643.575758
1,1,Nov,34,NaN,NaN,5320,NaN,11.0,5,NaN,...,0.0,0.0,0.000000,0.149369,0.220325,0.220325,0.272309,0.255791,181.435670,181.435670
2,2,Nov,34,NaN,NaN,5233,NaN,11.0,5,NaN,...,1.0,1.4,0.909091,0.149369,0.321789,0.272727,0.272309,0.255791,216.364304,199.757576
3,3,Nov,34,NaN,NaN,5232,NaN,11.0,5,NaN,...,0.0,0.2,0.090909,0.149369,0.080808,0.030303,0.272309,0.255791,47.752229,18.151515
4,4,Nov,34,NaN,NaN,5268,NaN,11.0,5,NaN,...,0.0,0.0,0.000000,0.149369,0.220325,0.220325,0.272309,0.255791,181.435670,181.435670


In [5]:
validation.head()

,Unnamed: 0,Month,date_block_num,item_category_id,item_cnt_month,item_id,item_price,m_num,shop_id,Revenue,...,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
0,0,Oct,33,0.0,0.0,5572,0.0,10.0,2,0.0,...,0.0,0.333333,0.416667,0.11552,1.136364,1.060606,0.272309,0.218626,1590.135895,1505.450000
1,1,Oct,33,0.0,0.0,5643,0.0,10.0,2,0.0,...,0.0,0.000000,0.833333,0.11552,1.785714,1.424242,0.272309,0.218626,4042.562819,3299.081818
2,2,Oct,33,0.0,0.0,5583,0.0,10.0,2,0.0,...,0.0,0.666667,0.416667,0.11552,0.547619,0.242424,0.272309,0.218626,364.164069,163.100000
3,3,Oct,33,0.0,0.0,7893,0.0,10.0,2,0.0,...,1.0,1.166667,1.666667,0.11552,2.975469,2.909091,0.272309,0.218626,5166.950289,5220.363636
4,4,Oct,33,6.0,4.0,7894,9160.0,10.0,2,36640.0,...,1.0,1.333333,1.333333,0.11552,3.562771,1.484848,0.272309,0.218626,4874.250863,2264.818182


Don't know what this column Unnamed is, but we will likely drop it.

In [6]:
# Drop
validation.drop(['Unnamed: 0'], axis=1, inplace=True)
test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
validation.head()

,Month,date_block_num,item_category_id,item_cnt_month,item_id,item_price,m_num,shop_id,Revenue,year,...,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
0,Oct,33,0.0,0.0,5572,0.0,10.0,2,0.0,2015,...,0.0,0.333333,0.416667,0.11552,1.136364,1.060606,0.272309,0.218626,1590.135895,1505.450000
1,Oct,33,0.0,0.0,5643,0.0,10.0,2,0.0,2015,...,0.0,0.000000,0.833333,0.11552,1.785714,1.424242,0.272309,0.218626,4042.562819,3299.081818
2,Oct,33,0.0,0.0,5583,0.0,10.0,2,0.0,2015,...,0.0,0.666667,0.416667,0.11552,0.547619,0.242424,0.272309,0.218626,364.164069,163.100000
3,Oct,33,0.0,0.0,7893,0.0,10.0,2,0.0,2015,...,1.0,1.166667,1.666667,0.11552,2.975469,2.909091,0.272309,0.218626,5166.950289,5220.363636
4,Oct,33,6.0,4.0,7894,9160.0,10.0,2,36640.0,2015,...,1.0,1.333333,1.333333,0.11552,3.562771,1.484848,0.272309,0.218626,4874.250863,2264.818182


In [8]:
test.head()

,Month,date_block_num,item_category_id,item_cnt_month,item_id,item_price,m_num,shop_id,Revenue,year,...,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
0,Nov,34,NaN,NaN,5037,NaN,11.0,5,NaN,2015,...,0.0,1.2,1.000000,0.149369,0.807359,0.393939,0.272309,0.255791,1287.410133,643.575758
1,Nov,34,NaN,NaN,5320,NaN,11.0,5,NaN,2015,...,0.0,0.0,0.000000,0.149369,0.220325,0.220325,0.272309,0.255791,181.435670,181.435670
2,Nov,34,NaN,NaN,5233,NaN,11.0,5,NaN,2015,...,1.0,1.4,0.909091,0.149369,0.321789,0.272727,0.272309,0.255791,216.364304,199.757576
3,Nov,34,NaN,NaN,5232,NaN,11.0,5,NaN,2015,...,0.0,0.2,0.090909,0.149369,0.080808,0.030303,0.272309,0.255791,47.752229,18.151515
4,Nov,34,NaN,NaN,5268,NaN,11.0,5,NaN,2015,...,0.0,0.0,0.000000,0.149369,0.220325,0.220325,0.272309,0.255791,181.435670,181.435670


maybe go back and change the train/validation split to rows 27 or 28 and not only leave 33 to validation. 

## Preparing Data

Drop Features

In [9]:
train.drop(['Month', 'item_category_id', 'item_price', 'Revenue'], axis=1, inplace=True)
validation.drop(['Month', 'item_category_id', 'item_price', 'Revenue'], axis=1, inplace=True)
test.drop(['Month', 'item_category_id', 'item_price', 'Revenue'], axis=1, inplace=True)
train.head()

,date_block_num,item_cnt_month,item_id,m_num,shop_id,year,max_item_price,shop_max_price,max_item_cnt,shop_max_cnt,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
0,0,9.0,5572,1.0,2,2013,18979.5,274298.571428,17.0,96.0,0.0,9.0,9.0,0.11552,1.136364,1.060606,0.154522,0.208116,1590.135895,1505.450000
1,0,1.0,5643,1.0,2,2013,35260.0,274298.571428,23.0,96.0,0.0,1.0,1.0,0.11552,1.785714,1.424242,0.154522,0.208116,4042.562819,3299.081818
2,0,2.0,5583,1.0,2,2013,5592.0,274298.571428,9.0,96.0,0.0,2.0,2.0,0.11552,0.547619,0.242424,0.154522,0.208116,364.164069,163.100000
3,0,3.0,7893,1.0,2,2013,42630.0,274298.571428,38.0,96.0,0.0,3.0,3.0,0.11552,2.975469,2.909091,0.154522,0.208116,5166.950289,5220.363636
4,0,1.0,7894,1.0,2,2013,31290.0,274298.571428,37.0,96.0,0.0,1.0,1.0,0.11552,3.562771,1.484848,0.154522,0.208116,4874.250863,2264.818182


In [10]:
test.head()

,date_block_num,item_cnt_month,item_id,m_num,shop_id,year,max_item_price,shop_max_price,max_item_cnt,shop_max_cnt,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
0,34,NaN,5037,11.0,5,2015,25990.000000,217310.0,23.000000,155.0,0.0,1.2,1.000000,0.149369,0.807359,0.393939,0.272309,0.255791,1287.410133,643.575758
1,34,NaN,5320,11.0,5,2015,6296.064344,217310.0,10.016115,155.0,0.0,0.0,0.000000,0.149369,0.220325,0.220325,0.272309,0.255791,181.435670,181.435670
2,34,NaN,5233,11.0,5,2015,7191.750000,217310.0,10.000000,155.0,1.0,1.4,0.909091,0.149369,0.321789,0.272727,0.272309,0.255791,216.364304,199.757576
3,34,NaN,5232,11.0,5,2015,4796.000000,217310.0,6.000000,155.0,0.0,0.2,0.090909,0.149369,0.080808,0.030303,0.272309,0.255791,47.752229,18.151515
4,34,NaN,5268,11.0,5,2015,6296.064344,217310.0,10.016115,155.0,0.0,0.0,0.000000,0.149369,0.220325,0.220325,0.272309,0.255791,181.435670,181.435670


Drop the first 6 months from the training data to give us less data to model on and because it is not as predictive.

In [11]:
# Drop the first 6 months
train = train.query('date_block_num >= 6').copy()
train.head()

,date_block_num,item_cnt_month,item_id,m_num,shop_id,year,max_item_price,shop_max_price,max_item_cnt,shop_max_cnt,prev_month_itm_cnt,6_mnth_mvg_avg,12_mnth_mvg_avg,shop_mean,item_mean,shop_item_mean,year_mean,month_mean,item_price_mean,shop_item_price_mean
1188421,6,1.0,5572,7.0,2,2013,18979.5,274298.571428,17.0,96.0,2.0,1.500000,2.571429,0.11552,1.136364,1.060606,0.154522,0.198165,1590.135895,1505.450000
1188422,6,3.0,5643,7.0,2,2013,35260.0,274298.571428,23.0,96.0,2.0,1.666667,1.571429,0.11552,1.785714,1.424242,0.154522,0.198165,4042.562819,3299.081818
1188423,6,0.0,5583,7.0,2,2013,5592.0,274298.571428,9.0,96.0,0.0,0.166667,0.428571,0.11552,0.547619,0.242424,0.154522,0.198165,364.164069,163.100000
1188424,6,6.0,7893,7.0,2,2013,42630.0,274298.571428,38.0,96.0,3.0,3.000000,3.000000,0.11552,2.975469,2.909091,0.154522,0.198165,5166.950289,5220.363636
1188425,6,1.0,7894,7.0,2,2013,31290.0,274298.571428,37.0,96.0,1.0,0.833333,0.857143,0.11552,3.562771,1.484848,0.154522,0.198165,4874.250863,2264.818182


## Modeling

### Train/Validation Split

In [12]:
# Prepare data for train and val
#train
X_train = train.drop(['item_cnt_month', 'date_block_num'], axis=1)
y_train = train['item_cnt_month']
#val
X_val = validation.drop(['item_cnt_month', 'date_block_num'], axis=1)
y_val = validation['item_cnt_month']
#test
X_test = test.drop(['item_cnt_month', 'date_block_num'], axis=1)

### Tree Based Models

**XGBoost**

We use XGBoostRegressor because this is more of a regression problem than a classification problem.

In [13]:
# XGBoost
my_model = XGBRegressor(n_estimators=100, max_depth = 8,
                        eta = 0.2, seed = 0, min_child_weight = 20, 
                        subsample = 0.7, colsample_bytree = 0.7)

my_model.fit(X_train, y_train, 
             early_stopping_rounds = 10,
             eval_set = [(X_val, y_val)],
             eval_metric = 'rmse',
             verbose=20)

xgb_pred = my_model.predict(X_val)
xgb_test = my_model.predict(X_test)

#Evaluation
# Find the mean squared error
xgb_score = np.sqrt(mean_squared_error(y_val,xgb_pred))         
print('RMSE: ', xgb_score)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[06:54:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:54:55] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:1.49
Will train until validation_0-rmse hasn't improved in 10 rounds.
[20]	validation_0-rmse:0.941635
[40]	validation_0-rmse:0.904331
[60]	validation_0-rmse:0.863073
[80]	validation_0-rmse:0.853404
[99]	validation_0-rmse:0.846806
RMSE:  0.845696614676334


We can get it to under 0.9 (around 0.86) with 100 estimators, max_depth of 8, and eta of 0.2. Could probably get higher with lower eta and more trees, but it would take very long.

**CatBoost**

In [14]:
# CatBoost
cat_model = CatBoostRegressor(iterations=300, depth = 8, verbose=False, random_seed=0)
cat_model.fit(X_train, y_train,
              eval_set = (X_val, y_val))

cat_pred = cat_model.predict(X_val)
#Eval
cat_score = np.sqrt(mean_squared_error(y_val, cat_pred))
print("RMSE: ", cat_score)

RMSE:  0.9378177788908244


**Random Forest**

In [15]:
# Random forest
n_est = 150
rf_model = RandomForestRegressor(n_estimators=n_est, max_depth=4, random_state=0)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_val)

#Evaluate
rf_score = np.sqrt(mean_squared_error(y_val, rf_pred))
print('RMSE: ', rf_score)

RMSE:  0.9822900245680359


### Clustering

**KNN (K-Nearest Neighbor)**

In [16]:
# KNN
# we set n_neigbors to how many features we think are correlated enough to be chosen as neibors
knn_model = KNeighborsRegressor(n_neighbors = 7)
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_val)

# Evaluate
knn_score = np.sqrt(mean_squared_error(y_val, knn_pred))
print('RMSE: ', knn_score)

RMSE:  1.2209248992678035


K-Nearest Neighbor is better for classification challenges. 

### Linear Models

**Linear Regression**

We use linear regression and not logistic regression because this is more of a regression problem than a classification problem. We are not separating items into two or more categories (categorical) we are predicting the sales (numerical). 

In [17]:
# Scaling
LR_scaler = MinMaxScaler()
LR_scaler.fit(X_train)
LR_train = LR_scaler.transform(X_train)
LR_val = LR_scaler.transform(X_val)

In [18]:
# Linear Regressor
# Use linear instead of logistic because variable is continuous not binary
lr_model = LinearRegression()
lr_model.fit(LR_train, y_train)
lr_pred = lr_model.predict(LR_val)
# Evaluation
lr_score = np.sqrt(mean_squared_error(y_val, lr_pred))
print('RMSE: ', lr_score)

RMSE:  1.033401819913438


### Model Evaluation

In [19]:
# Evaluate the models
models = pd.DataFrame({
    'Model': ['XGBoost', 'CatBoost',
              'Random Forest',
              'Linear Regression','KNN'],
    'Score': [xgb_score, cat_score,
              rf_score,
              lr_score, knn_score]
})

models.sort_values(by='Score', ascending=True)

,Model,Score
0,XGBoost,0.845697
1,CatBoost,0.937818
2,Random Forest,0.982290
3,Linear Regression,1.033402
4,KNN,1.220925


XGBoost gives us the best solution. 

### XGBoost Test Prediction

Make a submission using xgboost because it is the best performing model

In [20]:
# Prediction output normalized
preds = list(map(lambda x: min(20,max(x,0)), list(xgb_test)))
sub_df = pd.DataFrame({'ID':test.index,'item_cnt_month': preds })
sub_df.head()

,ID,item_cnt_month
0,0,1.178088
1,1,0.000000
2,2,0.987506
3,3,0.316113
4,4,0.000000


In [21]:
sub_df.to_csv('MV_submission.csv', index=False)